### A demo for a 2-dimensional generative model
_Alex Malz (NYU)_

This is a demo for a highly specific 2-D generative model, with terminology specific to photo-$z$ posteriors.
I will at some point extend this to N-D though

In [ ]:
testdict = {'a':1, 'b':2}

In [ ]:
if 'a' in testdict.keys():
    print(testdict['a'])

In [ ]:
import pzgen

In [ ]:
from pzgen.pspace import PSpace

In [ ]:
with open('demo.txt') as infile:
    lines = (line.split(' ') for line in infile)
    in_dict = {defn[0] : defn[1:][0].strip() for defn in lines}

In [ ]:
should_be_bool = ['intrinsic_scatter', 'catastrophic_outliers', 'systematic_bias']

In [ ]:
should_be_float = ['scatter_constant', 'outlier_fraction', 'bias_constant']

In [ ]:
import numpy as np
import csv
import pandas as pd
from scipy.stats import kde
from sklearn.neighbors import KernelDensity
import matplotlib as mpl
mpl.use('PS')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

In [ ]:
title = 18
label = 16
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'DejaVu Serif'
mpl.rcParams['axes.titlesize'] = title
mpl.rcParams['axes.labelsize'] = label
mpl.rcParams['figure.subplot.left'] = 0.2
mpl.rcParams['figure.subplot.right'] = 0.9
mpl.rcParams['figure.subplot.bottom'] = 0.2
mpl.rcParams['figure.subplot.top'] = 0.9
mpl.rcParams['figure.subplot.wspace'] = 0.5
mpl.rcParams['figure.subplot.hspace'] = 0.5

In [ ]:
points = pd.read_csv('jain05.csv', delimiter=',', names=['redshift', '"data"'])

In [ ]:
# remove outliers
to_use = points[np.logical_and(points['redshift'] > 0., points['"data"'] > 0)].values.T

In [ ]:
f = plt.figure(figsize=(5, 5))
plt.subplot(1, 1, 1)
plt.scatter(points['redshift'], points['"data"'], marker='.', s=1, color='k')
plt.plot([0, 4], [0, 4], color='r')
plt.xlabel(r'redshift')
plt.ylabel('"data"')
f.savefig('jain05.png', bbox_inches='tight', pad_inches=0, dpi=250)

In [ ]:
X, Y = np.linspace(0., 4., 100), np.linspace(0., 4., 100)
X, Y = np.meshgrid(X, Y)
xy = np.vstack([Y.ravel(), X.ravel()]).T
kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(to_use.T)

In [ ]:
# slow!
Z = np.exp(kde.score_samples(xy))

In [ ]:
# 2d KDE
f = plt.figure(figsize=(6, 5))
plt.subplot(1, 1, 1)
Z = Z.reshape(X.shape)
plt.contourf(X, Y, Z, cmap=plt.cm.viridis)
plt.colorbar()
plt.xlabel(r'redshift')
plt.ylabel('"data"')
f.savefig('jain05.png', bbox_inches='tight', pad_inches=0, dpi=250)

In [ ]:
for_posterior = points[np.logical_and(points['"data"'] > 2.9, points['"data"'] < 3.1)]['redshift'].values[:, np.newaxis]
for_likelihood = points[np.logical_and(points['redshift'] > 1.9, points['redshift'] < 2.1)]['"data"'].values[:, np.newaxis]

In [ ]:
plotgrid = np.linspace(0., 4., 100)

In [ ]:
kde_p = KernelDensity(kernel='gaussian', bandwidth=0.05).fit(for_posterior)
posterior = np.exp(kde_p.score_samples(plotgrid[:, np.newaxis]))
kde_l = KernelDensity(kernel='gaussian', bandwidth=0.05).fit(for_likelihood)
likelihood = np.exp(kde_l.score_samples(plotgrid[:, np.newaxis]))

In [ ]:
# 2d KDE
f, scatplot = plt.subplots(figsize=(8.5, 7.5))
f.subplots_adjust(hspace=0)
Z = Z.reshape(X.shape)
scatplot.contourf(Y, X, Z, cmap=plt.cm.viridis_r, alpha=0.75)
plt.plot([0, 4], [0, 4], color='r', alpha=0.25)
plt.scatter(points['redshift'], points['"data"'], marker='.', s=1, color='k', alpha=0.25)
# scatplot.colorbar()
scatplot.vlines(3., 0., 4., linewidth=10., alpha=0.5, color=plt.cm.cool(1.))
scatplot.hlines(2., 0., 4., linewidth=10., alpha=0.5, color=plt.cm.cool(0.))
scatplot.set_xlabel(r'redshift')
scatplot.set_ylabel('"data"')
scatplot.set_xlim(0., 4.)
scatplot.set_ylim(0., 4.)
scatplot.set_yticks([0., 1., 2., 3., 4])
scatplot.set_xticks([0., 1., 2., 3., 4])
divider = make_axes_locatable(scatplot)
histx = divider.append_axes('top', 1.2, pad=0., sharex=scatplot)
histy = divider.append_axes('right', 1.2, pad=0., sharey=scatplot)
histx.xaxis.set_tick_params(labelbottom=False)
histy.yaxis.set_tick_params(labelleft=False)
histx.plot(plotgrid, likelihood, color=plt.cm.cool(0.))
histy.plot(posterior, plotgrid, color=plt.cm.cool(1.))
histx.set_yticks([])
histy.set_xticks([])
histx.text(0.5, 1.25, 'posterior', rotation=0, size=16)
histy.text(0.75, 2., 'likelihood', rotation=-90, size=16)
f.savefig('jain05.png', bbox_inches='tight', pad_inches=0, dpi=250)